![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [170]:
# Start your code here!
import pandas as pd
import numpy as np

In [171]:
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs_clean = ds_jobs.copy()
print(ds_jobs_clean.dtypes)

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object


In [172]:
# Columns containing integers must be stored as 32-bit integers (int32).
int_variables=["student_id", "training_hours", "job_change"]
ds_jobs_clean[int_variables] = ds_jobs_clean[int_variables].astype("int32")

In [173]:
# Columns containing floats must be stored as 16-bit floats (float16).
float_variables=["city_development_index"]
ds_jobs_clean[float_variables] = ds_jobs_clean[float_variables].astype("float16")

In [174]:
# Columns containing nominal categorical data must be stored as the category data type.
obj_variables = [col for col in ds_jobs_clean.columns if ds_jobs_clean[col].dtype == "object"]
ds_jobs_clean[obj_variable] = ds_jobs_clean[obj_variable].astype("category")
print(ds_jobs_clean[obj_variable].dtypes)

city                   category
gender                 category
relevant_experience    category
enrolled_university    category
education_level        category
major_discipline       category
experience             category
company_size           category
company_type           category
last_new_job           category
dtype: object


In [175]:
# Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column
obj_variables = [col for col in ds_jobs_clean.columns if ds_jobs_clean[col].dtype == 'category']
for col in obj_variables:
    print(f"_{col.upper()}_".center(30, '*'))
    print(f'{col}: {ds_jobs_clean[col].unique()}', "\n")

************_CITY_************
city: ['city_103', 'city_40', 'city_21', 'city_115', 'city_162', ..., 'city_121', 'city_129', 'city_8', 'city_31', 'city_171']
Length: 123
Categories (123, object): ['city_1', 'city_10', 'city_100', 'city_101', ..., 'city_94', 'city_97', 'city_98', 'city_99'] 

***********_GENDER_***********
gender: ['Male', NaN, 'Female', 'Other']
Categories (3, object): ['Female', 'Male', 'Other'] 

****_RELEVANT_EXPERIENCE_*****
relevant_experience: ['Has relevant experience', 'No relevant experience']
Categories (2, object): ['Has relevant experience', 'No relevant experience'] 

****_ENROLLED_UNIVERSITY_*****
enrolled_university: ['no_enrollment', 'Full time course', NaN, 'Part time course']
Categories (3, object): ['Full time course', 'Part time course', 'no_enrollment'] 

******_EDUCATION_LEVEL_*******
education_level: ['Graduate', 'Masters', 'High School', NaN, 'Phd', 'Primary School']
Categories (5, object): ['Graduate', 'High School', 'Masters', 'Phd', 'Primary 

In [176]:
ordered_categories = {
    "relevant_experience" : ['Has relevant experience', 'No relevant experience'],
    "education_level" : ['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'],
    "experience" : ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    "company_size": ['<10', '10-49', '50-99','100-499','500-999', '1000-4999', '5000-9999', '10000+'],
    "last_new_job": ['1', '2', '3', '4', '>4', 'never'],
    "enrolled_university" : ['Full time course', 'Part time course', 'no_enrollment']
}

In [177]:
# Storing columns containing ordinal categorical data as ordered categories
for col in ds_jobs_clean:
    for col in ordered_categories.keys():
        category = pd.CategoricalDtype(categories=ordered_categories[col], ordered=True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)

# To order new DF columns as the original dataset
ds_jobs_clean = ds_jobs_clean[ds_jobs.columns]

In [178]:
# The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
